In [1]:
# Collecting Data
# Make a strategy and test a hypothesis 
# Backtest the Strategy 
#  Search for implementing thme in different markets
# Implement in Production
# Actual Trades are not implemented in the Code

In [2]:
#  We'll be using both Alpha Vantage APIs and IES Sandbox Cloud APIs
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install alpha_vantage


In [4]:
import numpy as np
import pandas  as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import requests 
# Make https requests
import os 
from copy import copy 
from scipy import math
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go


In [5]:
!pip install pyEX

In [6]:
import pyEX as p


In [26]:
api_key=open('/content/gdrive/MyDrive/DataSets/iexapitoken.txt').read()
apikey='sk_70d483dddb194e5ea39a5a4c62fdb3c3'


In [12]:
c = p.Client(api_token=apikey, version='stable')

In [24]:
sym='AAPL'
timeframe='5d'
df = c.chartDF(symbol=sym, timeframe=timeframe)[['close', 'volume']]

In [25]:
df

,close,volume
date,,
2021-07-22,146.80,77338156
2021-07-21,145.40,74993460
2021-07-20,146.15,96350036
2021-07-19,142.45,121434571


In [20]:
!pip install xlsxwriter



     |████████████████████████████████| 149 kB 5.2 MB/s 


In [21]:
import xlsxwriter
import math

In [22]:
sp=pd.read_csv('/content/gdrive/MyDrive/DataSets/sp_500_stocks.csv')

In [57]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'


In [58]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
# data=requests.get(api_url).json()
# data

In [59]:
data=requests.get(api_url).json()
# print(requests.get(api_url).content)
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'OG RMK GSA)STAQ/SEBC EALEDNALLNT(',
 'calculationPrice': 'close',
 'open': 0,
 'openTime': None,
 'openSource': 'icfflaoi',
 'close': 0,
 'closeTime': None,
 'closeSource': 'aflciifo',
 'high': 0,
 'highTime': None,
 'highSource': None,
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 151.6,
 'latestSource': 'Close',
 'latestTime': 'July 22, 2021',
 'latestUpdate': 1699167177228,
 'latestVolume': None,
 'iexRealtimePrice': 154,
 'iexRealtimeSize': 101,
 'iexLastUpdated': 1704698288670,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 152.3,
 'previousVolume': 75060142,
 'change': 1.4,
 'changePercent': 0.01006,
 'volume': None,
 'iexMarketPercent': 20.908668078259826,
 'iexVolume': 1002202,
 'avgTotalVolume': 911935

In [60]:
data['latestPrice']
data['marketCap']

2453674703911

In [61]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe
# This is the format desgined for the project

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [62]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe
# Appending a stock
# Similarly we'll get the data for all in sp_500

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,151.6,2453674703911,N/A


In [63]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in sp['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [64]:
final_dataframe
# Now we have the S&P stocks covered

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,157.80,47543721627,N/A
1,AAL,22.19,13924942505,N/A
2,AAP,212.98,13947744258,N/A
3,AAPL,147.20,2503444002088,N/A
4,ABBV,119.70,215957005926,N/A
...,...,...,...,...
500,YUM,122.59,35867377040,N/A
501,ZBH,162.77,34360506507,N/A
502,ZBRA,540.39,29530347633,N/A
503,ZION,52.82,8285512212,N/A


In [ ]:
# Batch API calls are one of the easiest ways to improve the performance of your code.

# This is because HTTP requests are typically one of the slowest components of a script.

# Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

In [65]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [67]:
symbol_groups = list(chunks(sp['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,154.30,46101197381,N/A
1,AAL,21.32,13629922108,N/A
2,AAP,212.12,13873152076,N/A
3,AAPL,147.10,2554247154884,N/A
4,ABBV,120.32,211590439759,N/A
...,...,...,...,...
500,YUM,121.61,36624533358,N/A
501,ZBH,163.82,34069886395,N/A
502,ZBRA,560.47,29798888064,N/A
503,ZION,51.03,8349335170,N/A


In [68]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:100000000


In [69]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe
# Equal Weightage is given and shares will be bought accordingly

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,154.30,46101197381,1283
1,AAL,21.32,13629922108,9287
2,AAP,212.12,13873152076,933
3,AAPL,147.10,2554247154884,1346
4,ABBV,120.32,211590439759,1645
...,...,...,...,...
500,YUM,121.61,36624533358,1628
501,ZBH,163.82,34069886395,1208
502,ZBRA,560.47,29798888064,353
503,ZION,51.03,8349335170,3880


In [70]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [72]:

background_color = '#0a0a27'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [73]:
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )

0

In [74]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [75]:
writer.save()